In [52]:
import numpy as np
import os
import pandas as pd
import hdbscan
import optuna
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.decomposition import PCA
from hdbscan.validity import validity_index
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score
from sklearn.metrics.pairwise import haversine_distances
import warnings
import plotly.express as px

# Ignorar warnings
warnings.filterwarnings("ignore")

os.chdir('c:/Users/admin/OneDrive/Documents/TrabajoTesis')


In [53]:
def consolidar_faenas_empresa(df_input, dist_km=5):
    """
    Agrupa faenas de la misma 'NombreEmpresa' que esten a menos de 'dist_km' entre si.
    Calcula el centroide geografico y promedia las variables numericas (tiempos).
    """
    print(f"--- Iniciando Consolidacion por Empresa (Radio: {dist_km} km) ---")
    
    # Identificar columnas numericas y categoricas
    cols_num = df_input.select_dtypes(include=[np.number]).columns.tolist()
    cols_cat = df_input.select_dtypes(exclude=[np.number]).columns.tolist()
    
    # Asegurar que Latitud y Longitud esten en la lista numerica para promediar
    if 'Latitud' not in cols_num: cols_num.append('Latitud')
    if 'Longitud' not in cols_num: cols_num.append('Longitud')

    dfs_consolidados = []
    
    # Iterar por cada empresa
    empresas = df_input['NombreEmpresa'].unique()
    
    for empresa in empresas:
        group = df_input[df_input['NombreEmpresa'] == empresa].copy()
        
        # Si la empresa tiene solo 1 faena, no se hace nada
        if len(group) == 1:
            dfs_consolidados.append(group)
            continue
            
        # Calcular matriz de distancias geograficas (Haversine)
        # Convertir a radianes para el calculo
        coords_rad = np.radians(group[['Latitud', 'Longitud']].values)
        
        # Radio de la tierra aprox 6371 km
        dist_matrix = haversine_distances(coords_rad) * 6371
        
        # Clusterizar internamente: si estan a menos de X km, son el mismo 'sub_id'
        clusterer = AgglomerativeClustering(
            n_clusters=None,
            distance_threshold=dist_km,
            metric='precomputed',
            linkage='complete' 
        )
        labels = clusterer.fit_predict(dist_matrix)
        group['sub_id'] = labels
        
        # Definir reglas de agregacion
        agg_rules = {}
        for c in cols_num:
            agg_rules[c] = 'mean' # Promedio para tiempos y coordenadas
        for c in cols_cat:
            agg_rules[c] = 'first' # Mantener el primer valor categorico
            
        # Limpiar reglas para evitar errores
        if 'sub_id' in agg_rules: del agg_rules['sub_id']
        if 'NombreEmpresa' in agg_rules: del agg_rules['NombreEmpresa']

        # Colapsar el grupo
        group_consol = group.groupby('sub_id').agg(agg_rules).reset_index(drop=True)
        group_consol['NombreEmpresa'] = empresa # Restaurar nombre
        
        dfs_consolidados.append(group_consol)

    # Unir todo
    df_final = pd.concat(dfs_consolidados, ignore_index=True)
    
    print(f"   Faenas Originales: {len(df_input)}")
    print(f"   Faenas Consolidadas: {len(df_final)}")
    print(f"   Reduccion: {len(df_input) - len(df_final)} puntos fusionados.")
    
    return df_final

def report_drop(before_cols, after_cols, label):
    removed = list(set(before_cols) - set(after_cols))
    print(f"\n {label}: removidas {len(removed)} columnas")
    if removed:
        print(f"   -> {removed}")
    print(f"   Columnas antes: {len(before_cols)} | despues: {len(after_cols)}")

def preparar_datos(df_region):
    """
    1. One-hot encoding
    2. Seleccion numerica
    3. Filtro: drop dummies con frecuencia extrema (<2% o >98%)
    4. Escalado robusto
    """
    # One Hot Encoding
    df_encoded = pd.get_dummies(df_region, columns=['ProvinciaFaena'], drop_first=True, dtype=int)
    
    # Seleccion inicial de numericas
    df_model = df_encoded.select_dtypes(include=[np.number]).dropna()
    
    # Identificar columnas dummies creadas
    dummy_cols = [c for c in df_model.columns if c.startswith("ProvinciaFaena_")]
    
    # Filtro de Frecuencia Extrema
    before = df_model.columns.tolist()
    low_freq_threshold = 0.02
    high_freq_threshold = 0.98
    
    drop_freq = []
    for col in dummy_cols:
        freq = df_model[col].mean()
        if freq < low_freq_threshold or freq > high_freq_threshold:
            drop_freq.append(col)
            
    if drop_freq:
        df_model = df_model.drop(columns=drop_freq)
        
    report_drop(before, df_model.columns.tolist(), "Filtro de Frecuencia")

    print(f" Dimensiones finales antes de escalar: {df_model.shape}")
    
    # Usamos RobustScaler para manejar outliers en tiempos de viaje
    scaler = RobustScaler()
    X = scaler.fit_transform(df_model.values)

    return X, df_model.columns.tolist(), df_model.index

def obtener_grupos_variables(feature_names):
    geo_idxs = [i for i, c in enumerate(feature_names) if c in ['Latitud', 'Longitud']]
    log_idxs = [i for i, c in enumerate(feature_names) if c.startswith('time_') or c.startswith('Tiempo_Prt_')]
    cat_idxs = [i for i, c in enumerate(feature_names) if c.startswith('ProvinciaFaena_')]
    other_idxs = [i for i in range(len(feature_names)) if i not in geo_idxs + log_idxs + cat_idxs]

    return geo_idxs, log_idxs, cat_idxs, other_idxs

In [54]:
def crear_objective(X_scaled, geo_idxs, log_idxs, cat_idxs, other_idxs, ranges, selection_method):

    def objective(trial):
        # Pesos
        w_geo = 1.0
        w_log = trial.suggest_float("w_log", *ranges['w_log'])
        w_cat = trial.suggest_float("w_cat", *ranges['w_cat'])
        w_oth = trial.suggest_float("w_oth", *ranges['w_oth'])

        Xw = X_scaled.copy()
        if geo_idxs: Xw[:, geo_idxs] *= w_geo
        if log_idxs: Xw[:, log_idxs] *= w_log
        if cat_idxs: Xw[:, cat_idxs] *= w_cat
        if other_idxs: Xw[:, other_idxs] *= w_oth

        # PCA
        max_comp = min(20, Xw.shape[1])
        n_comp = trial.suggest_int("n_components", 5, max_comp)
        Xp = PCA(n_components=n_comp, random_state=42).fit_transform(Xw)

        # HDBSCAN
        min_cluster = trial.suggest_int("min_cluster", *ranges['min_cluster'])
        min_samples = trial.suggest_int("min_samples", *ranges['min_samples'])

        clusterer = hdbscan.HDBSCAN(
            min_cluster_size=min_cluster,
            min_samples=min_samples,
            metric='euclidean',
            gen_min_span_tree=True,
            cluster_selection_method=selection_method
        ).fit(Xp)

        labels = clusterer.labels_
        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)

        if n_clusters < 8: return -1.0

        # Metricas
        try:
            dbcv = validity_index(Xp, labels)
            trial.set_user_attr("dbcv_raw", dbcv) 

            # Silhouette
            mask = labels != -1
            if mask.sum() > n_clusters:
                sil = silhouette_score(Xp[mask], labels[mask])
            else:
                sil = -1.0
            trial.set_user_attr("silhouette", sil)

        except:
            return -1.0

        # Penalizaciones al Score
        score = dbcv
        
        # Castigo por ruido excesivo
        ruido = (labels == -1).mean()
        if ruido > 0.25: 
            score -= 0.30
            
        # Castigo por exceso de clusters (atomizacion)
        if n_clusters > 60:
            score -= 0.4

        return score

    return objective

def entrenar_modelo_regional(df_region, nombre_zona, n_trials=30, param_ranges=None, selection_method='leaf'):

    defaults = {
        'w_log': (1.0, 3.5),
        'w_cat': (1.0, 2.5),
        'w_oth': (0.5, 1.5),
        'min_cluster': (15, 40), 
        'min_samples': (15, 30),
        'n_components': (5, 20) 
    }

    ranges = defaults.copy()
    if param_ranges:
        ranges.update(param_ranges)

    print(f"\n Optimizacion DBCV para {nombre_zona} | Metodo: {selection_method}")

    X_scaled, feature_names, valid_idx = preparar_datos(df_region)
    geo, log, cat, oth = obtener_grupos_variables(feature_names)

    objective = crear_objective(X_scaled, geo, log, cat, oth, ranges, selection_method)

    optuna.logging.set_verbosity(optuna.logging.WARNING) 
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

    best_trial = study.best_trial
    
    sil = best_trial.user_attrs.get("silhouette", "N/A")
    dbcv_raw = best_trial.user_attrs.get("dbcv_raw", "N/A") 

    print(f"   Mejor Score (Castigado): {study.best_value:.4f}")
    print(f"   DBCV Real (Sin castigo): {dbcv_raw}") 
    print(f"   Silhouette: {sil}")
    print(f"   Parametros: {study.best_params}")

    # Reconstruccion final del modelo
    bp = best_trial.params

    Xw = X_scaled.copy()
    if geo: Xw[:, geo] *= 1.0
    if log: Xw[:, log] *= bp['w_log']
    if cat: Xw[:, cat] *= bp['w_cat']
    if oth: Xw[:, oth] *= bp['w_oth']

    pca_final = PCA(n_components=bp['n_components'], random_state=42).fit_transform(Xw)

    final_clusterer = hdbscan.HDBSCAN(
        min_cluster_size=bp['min_cluster'],
        min_samples=bp['min_samples'],
        metric='euclidean',
        cluster_selection_method=selection_method
    ).fit(pca_final)

    return final_clusterer.labels_, valid_idx

In [55]:
def optimizar_fusion_zona(df_zona, col_analisis, zona_name, start_id=0):
    print(f"\n Optimizando Fusion (Aglomerativa) para: {zona_name}...")
    
    # Rango de epsilon para probar la fusion
    rango_eps = np.linspace(0.15, 0.40, 40)
    
    df_eval = df_zona[df_zona[col_analisis] != -2].copy()
    if len(df_eval) == 0:
        return {}, start_id, 0

    df_encoded = pd.get_dummies(df_eval, columns=['ProvinciaFaena'], drop_first=True, dtype=int)
    X_feat = df_encoded.select_dtypes(include=[np.number]).values
    X_scaled = RobustScaler().fit_transform(X_feat)
    
    unique_labels = set(df_eval[col_analisis]) - {-1}
    if not unique_labels:
        return {}, start_id, 0

    centroids = []
    label_map_list = []
    
    for label in unique_labels:
        data = df_eval[df_eval[col_analisis] == label][['Latitud', 'Longitud']].values
        centroid = data.mean(axis=0)
        centroids.append(centroid)
        label_map_list.append(label)
        
    centroids = np.array(centroids)
    
    best_score = -2.0
    best_eps = 0.0
    best_mapping = None
    best_n_clusters = 0
    
    for eps in rango_eps:
        agg_cluster = AgglomerativeClustering(
            n_clusters=None, 
            distance_threshold=eps, 
            metric='euclidean',
            linkage='complete' 
        )
        try:
            merged_ids = agg_cluster.fit_predict(centroids)
        except ValueError:
            merged_ids = np.zeros(len(centroids), dtype=int)

        mapping = {old: new for old, new in zip(label_map_list, merged_ids)}
        mapping[-1] = -1
        
        labels_temp = df_eval[col_analisis].map(mapping).values
        n_clust = len(set(labels_temp) - {-1})
        
        if n_clust > 1:
            try:
                score = validity_index(X_scaled, labels_temp, metric='euclidean')
            except:
                score = -1.0
        else:
            score = -1.0
            
        if score > best_score:
            best_score = score
            best_eps = eps
            best_mapping = mapping
            best_n_clusters = n_clust
            
    print(f"MEJOR {zona_name}: Radio {best_eps:.3f} grados | DBCV {best_score:.4f} | Clusters: {best_n_clusters}")
    
    final_map = {}
    if best_mapping:
        for old_id, new_id in best_mapping.items():
            if new_id == -1:
                final_map[old_id] = -1
            else:
                final_map[old_id] = new_id + start_id
    else:
        for i, old_id in enumerate(unique_labels):
            final_map[old_id] = i + start_id

    used_ids = [v for v in final_map.values() if v != -1]
    next_start = (max(used_ids) + 1) if used_ids else start_id
    
    return final_map, next_start, best_score

def calcular_metricas_zona(df_zona, col_cluster, nombre_zona):
    print(f"\n == Evaluando: {nombre_zona} ==")
    df_eval = df_zona[df_zona[col_cluster] != -2].copy()
    
    if len(df_eval) == 0:
        print("No hay datos en esta zona.")
        return

    df_encoded = pd.get_dummies(df_eval, columns=['ProvinciaFaena'], drop_first=True, dtype=int)
    X_eval = df_encoded.select_dtypes(include=[np.number]).values
    X_scaled = RobustScaler().fit_transform(X_eval)
    
    labels = df_eval[col_cluster].values
    n_clusters = len(set(labels) - {-1})
    
    # Silhouette
    mask_clustered = labels != -1
    if np.sum(mask_clustered) > 2 and n_clusters > 1:
        sil = silhouette_score(X_scaled[mask_clustered], labels[mask_clustered], metric='euclidean')
    else:
        sil = -1.0
        
    # DBCV 
    try:
        if n_clusters >= 1:
            dbcv = validity_index(X_scaled, labels, metric='euclidean')
        else:
            dbcv = 0.0
    except Exception as e:
        print(f"   Error DBCV: {e}")
        dbcv = 0.0

    print(f"   Total Clusters: {n_clusters}")
    print(f"   Ruido:          {(np.sum(labels == -1) / len(labels)):.1%}")
    print(f"   ---------------------------")
    print(f"   Silhouette:     {sil:.4f}")
    print(f"   DBCV:           {dbcv:.4f}")

In [56]:
# Cargar Datos
df = pd.read_csv('NeoModelos/minas_con_tiempos_puertos.csv') 
df = df[(df['RecursoPrimarioInstalacion'] == 'COBRE') | (df['RecursoMineroInstalacion'] == 'SALMUERA (LITIO)')] 

# --- PASO NUEVO: CONSOLIDACION POR EMPRESA ---
# Agrupamos faenas de la misma empresa en un radio de 5km
df = consolidar_faenas_empresa(df, dist_km=5)

--- Iniciando Consolidacion por Empresa (Radio: 5 km) ---
   Faenas Originales: 10834
   Faenas Consolidadas: 2835
   Reduccion: 7999 puntos fusionados.


In [57]:
# Definir columnas a eliminar
# Ahora si podemos borrar NombreEmpresa porque ya la usamos
cols_to_drop = ['RutEmpresa','NombreEmpresa','RecursoMineroInstalacion','TipoInstalacion',
                'TipoRecursoInstalacion','RecursoPrimarioInstalacion', 'ComunaFaena', 
                'NombreFaena', 'CategoriaFaena', 'IdFaena', 'ProvinciaInstalacion', 
                'ComunaInstalacion','NombreInstalacion','IdTipoInstalacion','IdInstalacion',
                'Norte','Este','Huso','Datum','IdEstado','Estado']

# Eliminar distancias (mantener tiempos)
cols_to_drop += [c for c in df.columns if c.startswith('dist_')]
df = df.drop(columns=cols_to_drop, errors='ignore')

# --- ELIMINACION POR CORRELACION ---
CORRELATION_THRESHOLD = 0.995
print(f"\n--- Columnas Redundantes (Corr > {CORRELATION_THRESHOLD}) ---")

df_numeric = df.select_dtypes(include=[np.number])
scaler_corr = RobustScaler()
df_scaled_corr = pd.DataFrame(scaler_corr.fit_transform(df_numeric), columns=df_numeric.columns)
corr_matrix = df_scaled_corr.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > CORRELATION_THRESHOLD)]

if len(to_drop) > 0:
    print(f"  {len(to_drop)} columnas redundantes para eliminar.")
    df = df.drop(columns=to_drop)
    print(f" Columnas eliminadas. Nueva dimension: {df.shape}")
else:
    print(" No se encontro redundancia excesiva.")


--- Columnas Redundantes (Corr > 0.995) ---
  42 columnas redundantes para eliminar.
 Columnas eliminadas. Nueva dimension: (2835, 18)


In [58]:
# --- DIVISION GEOGRAFICA (Norte/Sur) ---
REGIONES_NORTE = ['XV', 'I', 'II']
REGIONES_SUR   = ['III', 'IV', 'V', 'RM', 'VI', 'VII']

print(f"--- Separando Chile en 2 Modelos ---")
df_norte = df[df['RegionFaena'].isin(REGIONES_NORTE)].copy()
df_sur   = df[df['RegionFaena'].isin(REGIONES_SUR)].copy()

print(f"1. Modelo Norte (XV, I, II): {len(df_norte)} instalaciones")
print(f"2. Modelo Sur (III a VII):   {len(df_sur)} instalaciones")

--- Separando Chile en 2 Modelos ---
1. Modelo Norte (XV, I, II): 550 instalaciones
2. Modelo Sur (III a VII):   2284 instalaciones


In [59]:
# --- DIVISION GEOGRAFICA (Norte/Sur) ---
REGIONES_NORTE = ['XV', 'I', 'II']
REGIONES_SUR   = ['III', 'IV', 'V', 'RM', 'VI', 'VII']

print(f"--- Separando Chile en 2 Modelos ---")
df_norte = df[df['RegionFaena'].isin(REGIONES_NORTE)].copy()
df_sur   = df[df['RegionFaena'].isin(REGIONES_SUR)].copy()

print(f"1. Modelo Norte (XV, I, II): {len(df_norte)} instalaciones")
print(f"2. Modelo Sur (III a VII):   {len(df_sur)} instalaciones")

--- Separando Chile en 2 Modelos ---
1. Modelo Norte (XV, I, II): 550 instalaciones
2. Modelo Sur (III a VII):   2284 instalaciones


In [60]:
# Configuracion para el Norte 
config_norte = {
    'min_cluster': (30, 42),    
    'min_samples': (20, 40),    
}

# Configuracion para el Sur 
config_sur = {
    'min_cluster': (25, 40),    
    'min_samples': (20, 40),    
}

# Entrenar Norte
labels_norte, idx_norte = entrenar_modelo_regional(
    df_norte, 
    "ZONA NORTE", 
    n_trials=60, 
    param_ranges=config_norte,
    selection_method='leaf'  
)

# Entrenar Sur
labels_sur, idx_sur = entrenar_modelo_regional(
    df_sur, 
    "ZONA SUR", 
    n_trials=60, 
    param_ranges=config_sur,
    selection_method='leaf' 
)


 Optimizacion DBCV para ZONA NORTE | Metodo: leaf

 Filtro de Frecuencia: removidas 1 columnas
   -> ['ProvinciaFaena_PARINACOTA']
   Columnas antes: 21 | despues: 20
 Dimensiones finales antes de escalar: (550, 20)


Best trial: 0. Best value: -1: 100%|██████████| 60/60 [00:01<00:00, 44.02it/s]


   Mejor Score (Castigado): -1.0000
   DBCV Real (Sin castigo): N/A
   Silhouette: N/A
   Parametros: {'w_log': 2.211254062667527, 'w_cat': 1.2139889111912767, 'w_oth': 1.4696903431628847, 'n_components': 11, 'min_cluster': 39, 'min_samples': 32}

 Optimizacion DBCV para ZONA SUR | Metodo: leaf

 Filtro de Frecuencia: removidas 12 columnas
   -> ['ProvinciaFaena_TALCA', 'ProvinciaFaena_MAIPO', 'ProvinciaFaena_MELIPILLA', 'ProvinciaFaena_LOS ANDES', 'ProvinciaFaena_VALPARAISO', 'ProvinciaFaena_COLCHAGUA', 'ProvinciaFaena_TALAGANTE', 'ProvinciaFaena_CURICO', 'ProvinciaFaena_CORDILLERA', 'ProvinciaFaena_MARGA MARGA', 'ProvinciaFaena_QUILLOTA', 'ProvinciaFaena_SANTIAGO']
   Columnas antes: 36 | despues: 24
 Dimensiones finales antes de escalar: (2284, 24)


Best trial: 11. Best value: 0.768608: 100%|██████████| 60/60 [00:22<00:00,  2.68it/s]

   Mejor Score (Castigado): 0.7686
   DBCV Real (Sin castigo): 0.7686076680731141
   Silhouette: 0.7449770415640684
   Parametros: {'w_log': 1.0679248581209442, 'w_cat': 2.497717332861735, 'w_oth': 0.5207991177473419, 'n_components': 6, 'min_cluster': 36, 'min_samples': 36}


In [61]:
# --- UNIFICACION DE MODELOS ---
df['cluster_raw'] = -2 

# Asignar Norte 
df.loc[idx_norte, 'cluster_raw'] = labels_norte

# Asignar Sur (desplazando IDs para que no choquen)
max_id_norte = max(set(labels_norte) - {-1}) if len(set(labels_norte) - {-1}) > 0 else 0
offset = max_id_norte + 1
shifted_labels_sur = [x + offset if x >= 0 else -1 for x in labels_sur]
df.loc[idx_sur, 'cluster_raw'] = shifted_labels_sur

print("\n == Modelos unificados ==")
print(f"Total Clusters Brutos: {len(set(df['cluster_raw']) - {-1, -2})}")

# --- FUSION AGLOMERATIVA ---
col_analisis = 'cluster_raw'
col_final = 'cluster_final'

print("--- Separando zonas para fusion ---")
df_norte_final = df[df['RegionFaena'].isin(REGIONES_NORTE)]
df_sur_final = df[~df['RegionFaena'].isin(REGIONES_NORTE)]

mapa_norte, next_id, score_norte = optimizar_fusion_zona(df_norte_final, col_analisis, "ZONA NORTE", start_id=0)
mapa_sur, _, score_sur = optimizar_fusion_zona(df_sur_final, col_analisis, "ZONA SUR", start_id=next_id)

full_mapping = {**mapa_norte, **mapa_sur}
full_mapping[-1] = -1
full_mapping[-2] = -2

df[col_final] = df[col_analisis].map(full_mapping).fillna(-1).astype(int)


 == Modelos unificados ==
Total Clusters Brutos: 10
--- Separando zonas para fusion ---

 Optimizando Fusion (Aglomerativa) para: ZONA NORTE...
MEJOR ZONA NORTE: Radio 0.150 grados | DBCV 0.1037 | Clusters: 2

 Optimizando Fusion (Aglomerativa) para: ZONA SUR...
MEJOR ZONA SUR: Radio 0.150 grados | DBCV 0.3096 | Clusters: 8


In [62]:
# Evaluar Globalmente y por zonas
calcular_metricas_zona(df[df['RegionFaena'].isin(REGIONES_NORTE)], col_final, "ZONA NORTE (Final)")
calcular_metricas_zona(df[~df['RegionFaena'].isin(REGIONES_NORTE)], col_final, "ZONA SUR (Final)")
calcular_metricas_zona(df, col_final, "GLOBAL (Chile)")


 == Evaluando: ZONA NORTE (Final) ==
   Total Clusters: 2
   Ruido:          32.9%
   ---------------------------
   Silhouette:     0.5831
   DBCV:           0.1460

 == Evaluando: ZONA SUR (Final) ==
   Total Clusters: 8
   Ruido:          1.6%
   ---------------------------
   Silhouette:     0.4139
   DBCV:           0.3392

 == Evaluando: GLOBAL (Chile) ==
   Total Clusters: 10
   Ruido:          7.7%
   ---------------------------
   Silhouette:     0.4434
   DBCV:           0.3157


In [63]:
# Visualizacion en Mapa
plot_data = df[df[col_final] != -2].copy()
plot_data['Cluster_ID'] = plot_data[col_final].astype(str)
plot_data = plot_data.sort_values(col_final)

fig = px.scatter_mapbox(
    plot_data,
    lat="Latitud",
    lon="Longitud",
    color="Cluster_ID",
    color_discrete_map={'-1': 'lightgray'}, 
    hover_name="Cluster_ID",
    hover_data=["ProvinciaFaena"],
    zoom=5,
    center={"lat": -28.0, "lon": -69.5},
    title="Modelo Final: Consolidacion Empresa + Split Norte/Sur + Fusion",
    height=900
)

fig.update_layout(mapbox_style="carto-positron")
fig.show()